In [ ]:
!pip install pygame 

import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<Surface(640x480x32 SW)>

In [ ]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [ ]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
import gym
import random

#Red — 0 , Green — 1, Yellow — 2, and Blue — 3 for pick up
streets = gym.make("Taxi-v3").env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4

streets.reset()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


488

In [ ]:
##Red — 0 , Green — 1, Yellow — 2, and Blue — 3 for pick up
#Each state is defined by a 4 entries tuple: （taxi_row, taxi_col, passenger_location, destination)
initial_state = streets.encode(2, 1, 3, 1)

streets.s = initial_state

streets.render()

#State Space:  25 possible taxi positions, 5 possible locations of the passenger
# 25*5*4 = 500

/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


In [ ]:
streets.observation_space

Discrete(500)

In [ ]:
import numpy as np
#Action space:6 --> N,S,E,W, DROP-OFF, PICKUP
#Rewards: CORRECT FINAL DEST. +20, STEP -1, INCORRECT PICK/DROP -10

q_table = np.zeros([streets.observation_space.n, streets.action_space.n]) # 500 , 6
# a 2D array that represent every possible state and action in the virtual space and initialize all of them to 0
total_rewards = 0
learning_rate = 0.1
discount_factor = 0.5
exploration = 0.1
epochs = 5000

for taxi_run in range(epochs + 1): #Start training (the agent plays the number of epochs)
    state = streets.reset()
    done = False
    total_rewards = 0
    steps=0
    while not done:#each epoch/play contains this number of actions, starting from pickup a passenger until drop-off
        steps +=1
        random_value = random.uniform(0, 1)
        if (random_value < exploration):
            action = streets.action_space.sample() # Explore a random action
        else:
            action = np.argmax(q_table[state]) # Return the action with the highest q-value
            
        next_state, reward, done, info = streets.step(action) # Do the above action
        
        prev_q = q_table[state, action]
        next_max_q = np.max(q_table[next_state])
        # see RL-2 PPT file --- slide# 5
        new_q = (1 - learning_rate) * prev_q + learning_rate * (reward + discount_factor * next_max_q)
        total_rewards += reward
        q_table[state, action] = new_q
        #streets.render()        
        state = next_state

    if not(taxi_run % 1000):
      print(f'Epoch {taxi_run} Rewards={total_rewards} Steps={steps}')


Epoch 0 Rewards=-2616 Steps=1098
Epoch 1000 Rewards=-136 Steps=121
Epoch 2000 Rewards=-31 Steps=34
Epoch 3000 Rewards=0 Steps=12
Epoch 4000 Rewards=-13 Steps=25
Epoch 5000 Rewards=11 Steps=10


Epoch 0 Rewards=-1788 Steps=729 <br>
Epoch 1000 Rewards=-28 Steps=31 <br>
Epoch 2000 Rewards=-7 Steps=19 <br>
Epoch 3000 Rewards=6 Steps=15 <br>
Epoch 4000 Rewards=-5 Steps=17 <br>
Epoch 5000 Rewards=6 Steps=15 <br>

In [ ]:
range(epochs + 1)
exploration

0.1

In [ ]:
from IPython.display import clear_output
from time import sleep

lengths = []
rewards = []
number_of_games = 10
   
for tripnum in range(1, number_of_games + 1):
    state = streets.reset()
    done = False
    trip_length = 0
    game_rewards = 0

    while not done and trip_length < 30:
        action = np.argmax(q_table[state])
        next_state, reward, done, info = streets.step(action)
        game_rewards += reward
        clear_output(wait=True)
        print("Trip number " + str(tripnum) + " Step " + str(trip_length))
        print(streets.render(mode='ansi'))
        sleep(.2)
        state = next_state
        trip_length += 1

    lengths.append(trip_length)
    rewards.append(game_rewards)
    
    sleep(.2)

avg_steps = sum(lengths) / number_of_games
avg_rewards = sum(rewards) / number_of_games
print(f'Avg rewards is {avg_rewards} and Avg steps {avg_steps}')

Trip number 10 Step 29
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)

Avg rewards is -11.1 and Avg steps 21.6


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

